# Initialize

In [15]:
import numpy as np
import os
import pickle
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences as padding

# Data processing

In [22]:
def pickle_jar(filepath):
    file = open(filepath,'rb')
    thingtoload = pickle.load(file)
    file.close()
    return thingtoload
    
sequences = pickle_jar(os.path.join('sequences_output'))
labels = pickle_jar(os.path.join('label_output'))
label_map = pd.read_csv("/Users/jerremy/Desktop/python/hackathon/output_files/label_map.csv").iloc[0]

X =np.array(padding(sequences, padding="post"))
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [23]:
#TENSORBOARD GUI
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

#To use this, go to command prompt and navitage to logs file
#then go train folder
#then type 'tensorboard --logdir=.'
#then use the link

In [27]:
labels.shape

(69,)

# Model

In [30]:
#define and build model
model = Sequential(
    [
        Masking(mask_value=0.0,input_shape=(134,1662)),
        LSTM(64, return_sequences=True, activation='relu'),
        LSTM(128, return_sequences=True, activation='relu'),
        LSTM(64, return_sequences=False, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense((label_map.shape[0]-1), activation='softmax')
    ])

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_5 (Masking)         (None, 134, 1662)         0         
                                                                 
 lstm_15 (LSTM)              (None, 134, 64)           442112    
                                                                 
 lstm_16 (LSTM)              (None, 134, 128)          98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 7)                

In [31]:
#start training

#to continue training existing model, run this line
# model.load_weights('/Users/jerremy/Desktop/python/hackathon/ASL_model.h5')

model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000
3/3 [==============================] - 10s 663ms/step - loss: 1.9417 - categorical_accuracy: 0.1077
Epoch 2/2000
3/3 [==============================] - 2s 729ms/step - loss: 1.9233 - categorical_accuracy: 0.2308
Epoch 3/2000
3/3 [==============================] - 2s 735ms/step - loss: 1.9037 - categorical_accuracy: 0.2308
Epoch 4/2000
1/3 [=========>....................] - ETA: 1s - loss: 1.7721 - categorical_accuracy: 0.2812

KeyboardInterrupt: 

In [22]:
#saves model
model.save('/Users/jerremy/Desktop/python/hackathon/MODEL')

INFO:tensorflow:Assets written to: /Users/jerremy/Desktop/python/hackathon/MODEL/assets


INFO:tensorflow:Assets written to: /Users/jerremy/Desktop/python/hackathon/MODEL/assets


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

array([[[2., 0.],
        [1., 0.]],

       [[1., 1.],
        [1., 0.]],

       [[1., 2.],
        [0., 0.]],

       [[2., 0.],
        [1., 0.]]])

In [ ]:
accuracy_score(ytrue, yhat)

0.0